# Perform Object Detection With Mobilenet-SSD in OpenCV
This tutorial is based on the following website: https://www.pyimagesearch.com/2017/09/11/object-detection-with-deep-learning-and-opencv/

---
### Mount to Google Drive
Mount to your Google Drive to access the images and model files.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

dataset_location = "/content/drive/My Drive/Crafting/ADLCV/dataset/"

---
## Import of libraries

In [ ]:
# import the necessary packages
import numpy as np
import cv2
%matplotlib inline
import matplotlib.pyplot as plt

print(cv2.__version__)

---
## Load the model

In [ ]:
# load our serialized model from disk
print("[INFO] loading model...")
prototxt_file = dataset_location + "MobileNetSSD/MobileNetSSD_deploy.prototxt.txt"
model_file = dataset_location + "MobileNetSSD/MobileNetSSD_deploy.caffemodel"
net = cv2.dnn.readNetFromCaffe(prototxt_file, model_file)

---
### Load Image and prepare blob

In [ ]:
# load the input image and construct an input blob for the image
# by resizing to a fixed 300x300 pixels and then normalizing it
# (note: normalization is done via the authors of the MobileNet SSD
# implementation)
image_file = dataset_location + "MobileNetSSD/example_01.jpg"
image = cv2.imread(image_file)
(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)

# show the input image
image_display = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(15, 15))
plt.imshow(image_display)
plt.show()

---
### Perform object detection

In [ ]:
# pass the blob through the network and obtain the detections and
# predictions
print("[INFO] computing object detections...")
net.setInput(blob)
detections = net.forward()

---
### Loop through ```detections``` and draw bounding boxes

In [ ]:
# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]

# set the minimum confidence to accept the detection
minimum_confidence = 0.6

# loop over the detections
for i in np.arange(0, detections.shape[2]):
	# extract the confidence (i.e., probability) associated with the
	# prediction
	confidence = detections[0, 0, i, 2]
	# filter out weak detections by ensuring the `confidence` is
	# greater than the minimum confidence
	if confidence > minimum_confidence:
		# extract the index of the class label from the `detections`,
		# then compute the (x, y)-coordinates of the bounding box for
		# the object
		idx = int(detections[0, 0, i, 1])
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")
		# display the prediction
		label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
		print("[INFO] {}".format(label))
		cv2.rectangle(image, (startX, startY), (endX, endY),
			(0,0,0), 2)
		y = startY - 15 if startY - 15 > 15 else startY + 15
		cv2.putText(image, label, (startX, y),
			cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)

---
### Display the object detections

In [ ]:
# show the output image
image_display = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(15, 15))
plt.imshow(image_display)
plt.show()